In [6]:
import torch
import numpy as np
"""
发送长度为K的导频符号序列，其中发送符号x[i]从发送集S中等概率随机选取，接收端接受y[i]
训练数据即T[i]=(S[i],y[i])
收集多个i时刻的信道输入状态向量与信道输出（K个），从而得到训练数据集T=[T1,T2,...Ti]
"""

'\n发送长度为K的导频符号序列，其中发送符号x[i]从发送集S中等概率随机选取，接收端接受y[i]\n训练数据即T[i]=(S[i],y[i])\n收集多个i时刻的信道输入状态向量与信道输出（K个），从而得到训练数据集T=[T1,T2,...Ti]\n'

In [2]:
"""
Step1. 开始构建数据生成器，产生S[i]与y[i]的数据对
根据后面的看法，似乎所有的观测值有一个特定的集合，y[i]只会在特定集合内获取
最终的网络输出相当于成为了一个分类问题
"""
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NumbersDataset(Dataset):
    def __init__(self, low, high):
        self.samples = list(range(low, high))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        transmitter = self.samples[idx]
        receiver = transmitter + 1
        return transmitter, receiver

In [3]:
dataset = NumbersDataset(0,50)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
print(len(dataset))
print(dataset[5])
print(next(iter(dataloader)))

50
(5, 6)
[tensor([15, 11, 12, 14, 19, 43, 22, 17, 23,  4]), tensor([16, 12, 13, 15, 20, 44, 23, 18, 24,  5])]


In [33]:
"""
Step2. 开始构建两个全连接神经网络D和G，使参数满足标准高斯分布
所述的生成器G是一个全连接神经网络
"""
import torch.nn as nn
from torchsummary import summary

class G(nn.Module):
    def __init__(self, p):
        super(G, self).__init__()
        self.FC1 = nn.Sequential(nn.Linear(4,256), nn.LeakyReLU())
        self.FC2 = nn.Sequential(nn.Linear(256,512), nn.LeakyReLU())
        self.FC3 = nn.Sequential(nn.Linear(512,1024), nn.LeakyReLU())
        self.FC4 = nn.Linear(1024,p)
    def forward(self, s):
#         隐含层层数为Ng
        s = self.FC1(s)
        s = self.FC2(s)
        s = self.FC3(s)
        s = self.FC4(s)
        return nn.Softmax(s)

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.FC1a = nn.Sequential(nn.Linear(10,256), nn.LeakyReLU())
        self.FC1b = nn.Sequential(nn.Linear(4,256), nn.LeakyReLU())
        self.FC2 = nn.Sequential(nn.Linear(512,512), nn.LeakyReLU())
        self.FC3 = nn.Sequential(nn.Linear(512,256), nn.LeakyReLU())
        self.FC4 = nn.Sequential(nn.Linear(256,1), nn.Sigmoid())

    def forward(self, q, s):
        s1a = self.FC1a(q)
        s1b = self.FC1b(s)
        s = torch.cat((s1a,s1b),1)
        s = self.FC2(s)
        s = self.FC3(s)
        s = self.FC4(s)
        return s

In [22]:
'''
在这里要
1. 输出网络的结构(print,torch.summary两种)
2. 查看网络的现有每层参数
3. 构造数据，喂入网络，查看网络的输出
'''

g = G(4)
print(g)
for name,parameters in g.named_parameters():
    print(name,':',parameters.size())
#     print(parameters)

a = torch.ones(1,4)
print(a)
print(g.forward(a))

G(
  (FC1): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC2): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC3): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC4): Linear(in_features=1024, out_features=4, bias=True)
)
FC1.0.weight : torch.Size([256, 4])
FC1.0.bias : torch.Size([256])
FC2.0.weight : torch.Size([512, 256])
FC2.0.bias : torch.Size([512])
FC3.0.weight : torch.Size([1024, 512])
FC3.0.bias : torch.Size([1024])
FC4.weight : torch.Size([4, 1024])
FC4.bias : torch.Size([4])
tensor([[1., 1., 1., 1.]])
Softmax(dim=tensor([[-0.0199, -0.0062, -0.0274, -0.0791]], grad_fn=<AddmmBackward>))


In [36]:
d = D()
print(d)
b = torch.ones(1,10)
s = torch.ones(1,4)
print(s,b)
print(d.forward(b,s))

D(
  (FC1a): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC1b): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC3): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC4): Sequential(
    (0): Linear(in_features=256, out_features=1, bias=True)
    (1): Sigmoid()
  )
)
tensor([[1., 1., 1., 1.]]) tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0.4842]], grad_fn=<SigmoidBackward>)


In [39]:
g.cuda()
summary(g,(1,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]           1,280
         LeakyReLU-2               [-1, 1, 256]               0
            Linear-3               [-1, 1, 512]         131,584
         LeakyReLU-4               [-1, 1, 512]               0
            Linear-5              [-1, 1, 1024]         525,312
         LeakyReLU-6              [-1, 1, 1024]               0
            Linear-7                 [-1, 1, 4]           4,100
Total params: 662,276
Trainable params: 662,276
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 2.53
Estimated Total Size (MB): 2.55
----------------------------------------------------------------


In [24]:
'''
Step3. 对两个网络进行初始化，按照高斯分布初始化参数结构
'''

# 初始化函数接受一个初始化过的网络作为参数输入，将其参数重新初始化为高斯分布
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('FC') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

NetG = G(4)
NetD = D()

# 【标记】关于此处参数是否高斯分布生效需要进一步做验证
NetG.apply(weights_init)
NetD.apply(weights_init)


D(
  (FC1a): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC1b): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC2): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC3): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (FC4): Linear(in_features=256, out_features=1, bias=True)
)

In [25]:
for name,parameters in g.named_parameters():
    print(name,':',parameters)
#     print(parameters)


FC1.0.weight : Parameter containing:
tensor([[ 0.2476,  0.3361, -0.0224,  0.3665],
        [-0.3288, -0.0021, -0.1355,  0.1622],
        [-0.0845, -0.3688,  0.1261, -0.0074],
        ...,
        [ 0.0375,  0.4836, -0.1558, -0.3141],
        [-0.0047,  0.0259,  0.3597,  0.2185],
        [ 0.1458,  0.3071, -0.1539,  0.3910]], requires_grad=True)
FC1.0.bias : Parameter containing:
tensor([-0.1422,  0.1799,  0.1027, -0.1581,  0.4635, -0.0596,  0.1315,  0.1170,
        -0.1837, -0.0923,  0.3385, -0.1950,  0.4048,  0.2080,  0.2331, -0.0882,
        -0.3681, -0.0169,  0.4692, -0.0214,  0.2953,  0.1764,  0.2863, -0.1139,
         0.1676,  0.3971, -0.1315,  0.3523, -0.3366,  0.4494,  0.2553, -0.0963,
         0.4506,  0.3084, -0.4688,  0.3498, -0.2792,  0.0443, -0.1918,  0.0030,
        -0.2830,  0.1134, -0.2853, -0.3767,  0.0801,  0.3050, -0.0069,  0.0238,
         0.1729,  0.3697, -0.2299, -0.2623, -0.3445,  0.3331, -0.3351,  0.1995,
        -0.3477, -0.2402, -0.3698, -0.4327,  0.3517, -0.32

In [ ]:
'''
Step4. 分别定义两个网络的损失函数和优化方法，进行训练
'''
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
# 这里产生一个固定的噪声，每轮学习完成后都使用固定噪声来进行输出
# fixed_noise = torch.randn(25, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
lr = 0.0002
beta1 = 0.5
optimizerD = torch.optim.Adam(NetD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = torch.optim.Adam(NetG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
'''
Step5. 开始进行网络训练
首先对鉴别器进行训练，然后训练生成器
'''
# Training Loop

num_epochs = 100

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print("Starting Training Loop...")
# For each epoch
dataset = NumbersDataset(0, 50)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        NetD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = NetD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = NetG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = NetD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Add the gradients from the all-real and all-fake batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        NetG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = NetD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())




